# Function calls

We now look into what happens when a function annotated with `@s6.jit` invokes another function. 

In [1]:
import s6

def inc(x):
    return x + 1

@s6.jit
def compute(): 
    x = 0
    while x != 50000:
        x = inc(x)
    return x
          
print(compute(), s6.inspect(compute).is_compiled, s6.inspect(inc).is_compiled)

50000 False True


In the example above, only the `compute` function is annotated with `s6.jit`. However, this function calls `inc` many times, triggering S6 to optimize it, even though `compute` isn't optimized yet. This is because S6 doesn't compile all functions in the call stack immediately. A function is compiled based on 2 considerations: 

* how many byte code instructions S6 has gone through (specifically the `Oracle` in the implementation)
* whether this function is hot

S6 is setup to only consider compilation after seeing 10,000 byte code instructions. After processing this number of instructions, S6 determines if the function is hot. A function is hot if we have stopped in the function at least twice. This makes it so we don't optimize a function the code rarely calls, or is super short and thus won't see much of a speed up. So in the best case scenario we will compile a function after seeing 20,000 byte code instructions and stopping in the same function twice. 

The following loop should be enough to optimize `compute` as well.

In [2]:
for _ in range(100):
    compute()
print(s6.inspect(compute).is_compiled, s6.inspect(inc).is_compiled)

True True


The strongjit for both functions is below.

In [3]:
print(s6.inspect(compute).strongjit)

type_feedback @10 monomorphic, int#9
type_feedback @12 monomorphic, bool#11
type_feedback @14 monomorphic, globals(__main__)#5870
type_feedback @18 monomorphic, function#401

function compute {
&0:                                                         // entry point
  %1 = constant $0
  bytecode_begin @0 fastlocals [%1]                         // LOAD_CONST 580800589.py:8
  %3 = frame_variable consts, $1
  incref notnull %3
  jmp &6 [ %3 ]

&6: [ %7 ]                                                  // preds: &0, &37
  bytecode_begin @6 fastlocals [%7] {kLoop @26 $0}          // LOAD_FAST 580800589.py:9
  %9 = frame_variable consts, $2
  %10 = unbox long %7
  %11 = overflowed? %10
  deoptimize_if_safepoint %11, @10 stack [%7, %9] fastlocals [%7] increfs [%7, %9] {kLoop @26 $0}, "Value was not unboxable" // COMPARE_OP 580800589.py:9
  %13 = constant $50000
  %14 = cmp ne i64 %10, %13
  advance_profile_counter $7
  br %14, &17, &33

&17:                                                 

Here's some code worth noting:

```
%19 = frame_variable globals, $0
%20 = get_instance_class_id %19
...
%24 = constant_attribute "inc" of globals(__main__)#5870
...
%29 = call_python fast %24 (%7) @18                       // CALL_FUNCTION 580800589.py:10
```

First, we've realised that the globals we're bound with have class-like semantics so we can do a class check on them (with `%19 = ...` and `%20 = ...`). Then, in `%24 = ...`, we know that the attribute `"inc"` is constant, and therefore S6 can use a fast jump to generated code, if it already exists, rather than going via the `EvalFrame` dispatch loop. This call happens in `%29 = ...`. Notice that it's calling `%24`, which is a `constant_attribute`, so we know the call target! The `fast` flag tells S6 it can use the fastest calling convention.

Something else to note is the deoptimized block:
```
&42: deoptimized                                            // preds: &17
  decref notnull %7 @18                                     // CALL_FUNCTION 4045191662.py:10
  except @18                                                // CALL_FUNCTION 4045191662.py:10
}
```
which is code that S6 didn't generate code for. It's accessible by a branch-like instruction `deoptimize_if`. If we hit the `true` branch we must jump out to the strongjit interpreter and interpret the deoptimized blocks until we get to a safepoint.

In [4]:
print(s6.inspect(inc).strongjit)

type_feedback @4 monomorphic, int#9

function inc {
&0: [ %1 ]                                                  // entry point
  bytecode_begin @0 fastlocals [%1]                         // LOAD_FAST 580800589.py:4
  %3 = frame_variable consts, $1
  %4 = unbox long %1
  %5 = overflowed? %4
  deoptimize_if_safepoint %5, @4 stack [%1, %3] fastlocals [%1] increfs [%3, %1], "Value was not unboxable" // BINARY_ADD 580800589.py:4
  %7 = constant $1
  %8 = add i64 %4, %7
  %9 = overflowed? %8
  deoptimize_if_safepoint %9, @4 stack [%1, %3] fastlocals [%1] increfs [%3, %1], "Value was not unboxable" // BINARY_ADD 580800589.py:4
  %11 = box long %8
  advance_profile_counter $4
  decref notnull %1 @6                                      // RETURN_VALUE 580800589.py:4
  return %11
}
